In [ ]:
import numpy as np
import cv2

%matplotlib inline
from matplotlib import pyplot as plt

np.random.seed(42)

In [ ]:
#Routine to fix 
def fixColor(image):
    return(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


In [ ]:
#Take a look at the input video
from IPython.display import Video
#Video("images/overpass.mp4", embed=True)

#Replace with your own video

In [ ]:
video_stream = cv2.VideoCapture('../input/mindnew/IMG_0425.MOV')

# Randomly select 30 frames

frameIds = video_stream.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=30)

# Store selected frames in an array
frames = []
for fid in frameIds:
    video_stream.set(cv2.CAP_PROP_POS_FRAMES, fid)
    ret, frame = video_stream.read()
    frames.append(frame)
    
video_stream.release()


In [ ]:
# Calculate the median along the time axis
medianFrame = np.median(frames, axis=0).astype(dtype=np.uint8)
plt.imshow(fixColor(medianFrame))

In [ ]:
# Calculate the average along the time axis
avgFrame = np.average(frames, axis=0).astype(dtype=np.uint8)
plt.imshow(fixColor(avgFrame))

In [ ]:
sample_frame=frames[0]
plt.imshow(fixColor(sample_frame))

For identifying moving objects it is better to work with grayscale images. We will convert both the median image and sample image to grayscale

In [ ]:
grayMedianFrame = cv2.cvtColor(medianFrame, cv2.COLOR_BGR2GRAY)
plt.imshow(fixColor(grayMedianFrame))

In [ ]:
graySample=cv2.cvtColor(sample_frame, cv2.COLOR_BGR2GRAY)
plt.imshow(fixColor(graySample))

## Remove background
Remove the background from our sample. We can now see a ghost image with cars and background removed.

In [ ]:
dframe = cv2.absdiff(graySample, grayMedianFrame)
plt.imshow(fixColor(dframe))

## Blurring

We will run Gaussian blurring to reduce noise and enable easier identification of edges

In [ ]:
blurred = cv2.GaussianBlur(dframe, (11,11), 0)
plt.imshow(fixColor(blurred))

In [ ]:
h,w= np.shape(blurred)
for px in range(0,h):
    for py in range(0,w):
         current_color = blurred[px][py]
         if(current_color<10):
            blurred[px][py]=0
            
         

## Thresholding

We will now run a threshold to clear bring out the objects left here. We will use OTSU thresholding which automatically figure out the correct thresold levels.

In [ ]:
plt.imshow(blurred)

## Contour and Bounding Boxes

We will create contours using the thresholded pictures

In [ ]:
(cnts, _) = cv2.findContours(tframe.copy(), cv2.RETR_EXTERNAL, 
                             cv2 .CHAIN_APPROX_SIMPLE)

We will now create bounding boxes for the contours identified and show them on our sample images. We will disregard items like moving clouds in the top of our picture

In [ ]:
for cnt in cnts:
    x,y,w,h = cv2.boundingRect(cnt)
    if y > 200:  #Disregard item that are the top of the picture
        cv2.rectangle(sample_frame,(x,y),(x+w,y+h),(0,255,0),2)

plt.imshow(fixColor(sample_frame))

In [ ]:
vid = cv2.VideoCapture("../input/mindnew/IMG_0425.MOV") 

currentframe = 0
while(True): 
      
    ret,frame = vid.read() 
  
    if ret: 
        name = 'frame' + str(currentframe) + '.jpg' 
        cv2.imwrite(name, frame) 
        currentframe += 1
    else: 
        print("--> Frames Created <--")
        break

In [ ]:
h,w= np.shape(blurred)
for px in range(0,h):
    for py in range(0,w):
         current_color = blurred[px][py]
         if(current_color<10):
            blurred[px][py]=0
            

In [ ]:
pip install Image

In [ ]:
from PIL import Image
import glob
#for filename in glob.glob('*.jpg'):
img = Image.open('../input/image1/Sample2.jpg')
height,width= img.size
for y in range(width):
    for x in range(height):
        current_color = img.getpixel( (x,y) )
        a,b,c=current_color
        new_color=(255,255,255)
        new_color1=(0,0,0)
        if(a>120 or b>120 or c>120):
            img.putpixel( (x,y), new_color)
        else:
            img.putpixel( (x,y), new_color1)
plt.imshow(img)    
    

In [ ]:
import pytesseract
text=pytesseract.image_to_string(img)
print("Text in Image :-")
print("----------------")
print(text)

print("Original Image :-")
print("----------------")
print(plt.imshow(img))

In [ ]:
height, width, layers = frames[0].shape
size = (width,height)

In [ ]:
out = cv2.VideoWriter('Out-Put.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

In [ ]:
import glob
img_array = []
for filename in glob.glob('*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    gframe = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   
    dframe = cv2.absdiff(gframe, grayMedianFrame)
  
    blurred = cv2.GaussianBlur(dframe, (11, 11), 0)
   
    ret, tframe= cv2.threshold(blurred,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  
    (cnts, _) = cv2.findContours(tframe.copy(),cv2.RETR_EXTERNAL, cv2 .CHAIN_APPROX_SIMPLE)

    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
        if y > 200:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    
    img_array.append(frame)
    
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

print("Output Video Created As Out-Put.avi")